In [36]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.stats import loguniform
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline

In [62]:
csv_data = pd.read_csv('C:/Users/暴脾气の尧/Desktop/#TUM 2 SEMESTER/Applied Machine Intelligence/elec_forecast.csv')
#C:/Users/U388316/OneDrive - Danfoss/Desktop/usb_tum/ami/Han-20210616T220428Z-001/Han/elec_forecast.csv
csv_data.head()

,Date,Time of day,Biomass[MWh],Hydropower[MWh],Wind offshore[MWh],Wind onshore[MWh],Photovoltaics[MWh],Other renewable[MWh],Nuclear[MWh],Fossil brown coal[MWh],...,Residual load[MWh],Lufttemp.+200cm,Lufttemp.+20cm,Bodentemp.-5cm,Bodentemp.-20cm,Luftfeuchtigkeit+200cm,Windgeschw.+250cm,Niederschlag,Globalstrahlg.+200cm,Spot price
0,2015/1/1,0,1006.25,289.75,129.25,2032.00,0.0,33.00,2677.75,3922.00,...,1380.75,2.9,2.0,3.1,3.4,100.0,1.5,0.0,39.0,25.02
1,2015/1/1,1,995.75,297.00,129.00,2074.25,0.0,30.75,2771.50,3830.75,...,1229.75,2.9,2.0,3.1,3.4,100.0,1.5,0.0,39.0,18.29
2,2015/1/1,2,1005.00,284.75,128.50,2135.25,0.0,23.50,2756.50,3704.50,...,1126.25,2.9,2.0,3.1,3.4,100.0,1.5,0.0,39.0,16.04
3,2015/1/1,3,1010.50,280.50,129.75,2138.25,0.0,22.00,2757.00,3518.75,...,1141.00,2.9,2.0,3.1,3.4,100.0,1.5,0.0,39.0,14.60
4,2015/1/1,4,1009.75,278.00,130.00,2161.00,0.0,22.00,2740.75,3529.00,...,1170.25,2.9,2.0,3.1,3.4,100.0,1.5,0.0,39.0,14.95


In [76]:
csv_data =csv_data[['Time of day','Wind onshore[MWh]','Fossil gas[MWh]','Hydro pumped storage[MWh]','Spot price']]
csv_data.head()

,Time of day,Wind onshore[MWh],Fossil gas[MWh],Hydro pumped storage[MWh],Spot price
0,0,2032.00,306.50,453.00,25.02
1,1,2074.25,217.75,272.50,18.29
2,2,2135.25,202.25,238.00,16.04
3,3,2138.25,205.25,141.75,14.60
4,4,2161.00,207.75,173.25,14.95


In [64]:
temp = csv_data['Date'].values.tolist()
index_split_train_test = []
for i in range(len(temp)):
    if temp[i] == '2020/12/31':
        index_split_train_test.append(i)
print(index_split_train_test[len(index_split_train_test)-1]) 

52595


We have 24 rows for one day,herefore, print the last element of index

In [65]:
print(temp[52594])
print(temp[52595]) # index to seperate
print(temp[52596])
print(temp[52597])

2020/12/31
2020/12/31
2021/1/1
2021/1/1


In [78]:
csv_data.isnull().sum()

Time of day                  0
Wind onshore[MWh]            0
Fossil gas[MWh]              0
Hydro pumped storage[MWh]    0
Spot price                   0
dtype: int64

In [79]:
csv_data = csv_data.fillna(csv_data.mean())
csv_data.isnull().sum()

Time of day                  0
Wind onshore[MWh]            0
Fossil gas[MWh]              0
Hydro pumped storage[MWh]    0
Spot price                   0
dtype: int64

In [81]:
train = csv_data[:52595]
test = csv_data[52596:]

#train = train.drop(['Date'],axis=1)
#test = test.drop(['Date'],axis=1)
                    
X_train = train.iloc[:, :-1]
Y_train = train.iloc[:, -1:]
X_test = test.iloc[:, :-1]
Y_test = test.iloc[:, -1:]
scx = StandardScaler()
scy = StandardScaler()
cv = KFold(n_splits=10,shuffle=True,random_state=1)

In [84]:
#如果用岭回归
from sklearn.linear_model import Ridge
reg = Ridge()
space = dict()
space['regressor__regression__solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['regressor__regression__alpha'] = [1e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1,5e-1, 1,2,5,10,20,50, 100, 200]

In [82]:
#如果用随机森林
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor()
space = dict() 
space['regressor__regression__n_estimators'] = [30,50,70,100]
space['regressor__regression__max_depth'] = [3,5,10,15,20]

In [85]:
pipe = Pipeline([('scaler', scx), 
                 ('regression', reg)])
model = TransformedTargetRegressor(regressor=pipe, transformer=scy) #注意scx，reg，pipe，scy都可以自己设置
gs = GridSearchCV(model, space, scoring='neg_root_mean_squared_error', n_jobs=1, cv=cv, refit=True) #n_jobs=-1更快但有可能会报错
gs.fit(X_train,Y_train)
print("Best CV-RMSE: {:.2f}".format(-gs.best_score_)) #print结果
print(gs.best_params_)

Y_pred_train = gs.predict(X_train)
Y_pred_test = gs.predict(X_test)
print("Training RMSE: {:.2f}".format(np.sqrt(mean_squared_error(Y_train, Y_pred_train)))) 
print("Test RMSE: {:.2f}".format(np.sqrt(mean_squared_error(Y_test, Y_pred_test))))
print("Training R2: {:.2f}".format(r2_score(Y_train, Y_pred_train))) 
print("Test R2: {:.2f}".format(r2_score(Y_test, Y_pred_test)))

Best CV-RMSE: 11.29
{'regressor__regression__alpha': 0.001, 'regressor__regression__solver': 'sag'}
Training RMSE: 11.29
Test RMSE: 17.67
Training R2: 0.45
Test R2: 0.12
